# Skip-Gram+NGE数据处理模块


**目录：**
1. 词表构建

2. 正样本生成

3. 负样本采样

4. 负样本生成

---

In [20]:
import numpy as np
from collections import deque

In [40]:
input_file = open("./data/text8.txt","r",encoding="utf-8")

In [41]:
def _init_dict(min_count):
    word_count_sum = 0
    sentence_count = 0
    word2id_dict = dict()
    id2word_dict = dict()
    wordid_frequency_dict = dict()
    word_freq = dict()
    for line in input_file:
        line = line.strip().split()
        word_count_sum +=len(line)
        sentence_count +=1
        for i,word in enumerate(line):
            if i%1000000==0:
                print (i,len(line))
            if word_freq.get(word)==None:
                word_freq[word] = 1
            else:
                word_freq[word] += 1
    for i,word in enumerate(word_freq):
        if i % 100000 == 0:
            print(i, len(word_freq))
            # 如果小于min_count就将他视为比较稀有的词，忽略
        if word_freq[word]<min_count:
            # 总的词频减去这个词的词频
            word_count_sum -= word_freq[word]
            continue
        word2id_dict[word] = len(word2id_dict) # 按照顺序下去，第几个进入的词分配第几
        id2word_dict[len(id2word_dict)] = word # 同样是按照顺序排下去
        wordid_frequency_dict[len(word2id_dict)-1] = word_freq[word]
    word_count =len(word2id_dict) # 去重之后词数量
    return word2id_dict,id2word_dict,wordid_frequency_dict

In [42]:
word2id_dict,id2word_dict,wordid_frequency_dict = _init_dict(3)

0 17005207
1000000 17005207
2000000 17005207
3000000 17005207
4000000 17005207
5000000 17005207
6000000 17005207
7000000 17005207
8000000 17005207
9000000 17005207
10000000 17005207
11000000 17005207
12000000 17005207
13000000 17005207
14000000 17005207
15000000 17005207
16000000 17005207
17000000 17005207
0 253854
100000 253854
200000 253854


In [43]:
word2id_dict

{'certifying': 0,
 'evaders': 1,
 'bilabial': 2,
 'fealty': 3,
 'joeseph': 4,
 'yozei': 56307,
 'obliterating': 5,
 'vyacheslav': 49833,
 'hangout': 6,
 'customizable': 49834,
 'uruguayan': 7,
 'cltl': 8,
 'die': 9,
 'tabasco': 10,
 'mayflower': 11,
 'rooted': 12,
 'bloomers': 49835,
 'bonnett': 13,
 'supercalifragilisticexpialidocious': 14,
 'lattuada': 49836,
 'unpretentious': 15,
 'euglenids': 49837,
 'magnolias': 83414,
 'aldohexose': 49839,
 'chandarli': 16,
 'confederaci': 49841,
 'fatalistic': 49842,
 'tongues': 17,
 'anwar': 91628,
 'independencia': 77,
 'progressives': 49844,
 'envied': 49845,
 'gao': 18,
 'gyro': 19,
 'immersions': 55197,
 'wrongdoings': 107,
 'propitious': 49847,
 'sanding': 16859,
 'prospered': 20,
 'goddamn': 49848,
 'pueblos': 49849,
 'enciclopedia': 21,
 'gelobet': 22,
 'lutterworth': 23,
 'carthusians': 89219,
 'doubs': 24,
 'editio': 49852,
 'iahp': 49853,
 'assurbanipal': 26,
 'tora': 27,
 'liberale': 28,
 'stevie': 29,
 'madhyamika': 148,
 'adiation'

In [44]:
def get_wordId_list():
    input_file = open("./data/text8.txt", encoding="utf-8")
    sentence = input_file.readline()
    wordId_list = []  # 一句中的所有word 对应的 id
    sentence = sentence.strip().split(' ')
    for i,word in enumerate(sentence):
        if i%1000000==0:
            print (i,len(sentence))
        try:
            word_id = word2id_dict[word]
            wordId_list.append(word_id)
        except:
            continue
    return wordId_list

In [45]:
wordId_list = get_wordId_list()

0 17005207
1000000 17005207
2000000 17005207
3000000 17005207
4000000 17005207
5000000 17005207
6000000 17005207
7000000 17005207
8000000 17005207
9000000 17005207
10000000 17005207
11000000 17005207
12000000 17005207
13000000 17005207
14000000 17005207
15000000 17005207
16000000 17005207
17000000 17005207


In [46]:
wordId_list

[34082,
 30219,
 56435,
 56350,
 42682,
 78940,
 22308,
 2027,
 67152,
 5683,
 12313,
 46866,
 37768,
 80120,
 18872,
 28271,
 54429,
 78940,
 28271,
 60778,
 5495,
 37681,
 28271,
 37442,
 74799,
 78940,
 28271,
 64204,
 5495,
 68444,
 28271,
 42682,
 97571,
 29682,
 67152,
 71001,
 56350,
 24664,
 5853,
 86269,
 58494,
 61294,
 37179,
 80804,
 67152,
 62608,
 71231,
 86269,
 92922,
 28271,
 91966,
 78940,
 27710,
 10356,
 15157,
 76868,
 47830,
 78221,
 65253,
 56435,
 56350,
 92639,
 38942,
 36741,
 91737,
 7021,
 51221,
 28271,
 68737,
 34082,
 97571,
 88432,
 70143,
 28271,
 20413,
 8024,
 99432,
 39498,
 76222,
 63652,
 34082,
 56435,
 56350,
 77624,
 44197,
 97571,
 28271,
 72514,
 80804,
 55821,
 40751,
 85830,
 37681,
 21389,
 86015,
 41220,
 50823,
 52307,
 40751,
 13484,
 6025,
 78940,
 60236,
 4639,
 71231,
 34082,
 76868,
 82650,
 86269,
 29432,
 93761,
 17925,
 80804,
 72866,
 28271,
 38486,
 78940,
 29998,
 346,
 4416,
 28271,
 22719,
 28271,
 68737,
 65868,
 56435,
 567

In [47]:

def get_batch_pairs(batch_size,window_size,index,word_pairs_queue):
    while len(word_pairs_queue) < batch_size:
        if index==len(wordId_list):
            index = 0
        for _ in range(1000):
            for i in range(max(index - window_size, 0),min(index + window_size + 1,len(wordId_list))):
                wordId_w = wordId_list[index]
                wordId_v = wordId_list[i]
                if index == i:  # 上下文=中心词 跳过
                    continue
                word_pairs_queue.append((wordId_w, wordId_v))
            index+=1
    result_pairs = []  # 返回mini-batch大小的正采样对
    for _ in range(batch_size):
        result_pairs.append(word_pairs_queue.popleft())
    return result_pairs

In [48]:
index = 0
word_pairs_queue = deque()
result_pairs = get_batch_pairs(32,3,index,word_pairs_queue)

In [49]:
result_pairs

[(34082, 30219),
 (34082, 56435),
 (34082, 56350),
 (30219, 34082),
 (30219, 56435),
 (30219, 56350),
 (30219, 42682),
 (56435, 34082),
 (56435, 30219),
 (56435, 56350),
 (56435, 42682),
 (56435, 78940),
 (56350, 34082),
 (56350, 30219),
 (56350, 56435),
 (56350, 42682),
 (56350, 78940),
 (56350, 22308),
 (42682, 30219),
 (42682, 56435),
 (42682, 56350),
 (42682, 78940),
 (42682, 22308),
 (42682, 2027),
 (78940, 56435),
 (78940, 56350),
 (78940, 42682),
 (78940, 22308),
 (78940, 2027),
 (78940, 67152),
 (22308, 56350),
 (22308, 42682)]

In [52]:
def _init_sample_table():
    sample_table = []
    sample_table_size = 1e8
    pow_frequency = np.array(list(wordid_frequency_dict.values())) ** 0.75
    word_pow_sum = sum(pow_frequency)
    ratio_array = pow_frequency / word_pow_sum
    word_count_list = np.round(ratio_array * sample_table_size)
    for word_index, word_freq in enumerate(word_count_list):
        sample_table += [word_index] * int(word_freq)
    sample_table = np.array(sample_table)
    return sample_table

In [53]:
sample_table = _init_sample_table()

In [54]:
sample_table

array([     0,      0,      0, ..., 100037, 100037, 100037])

In [55]:
def get_negative_sampling(positive_pairs, neg_count):
    neg_v = np.random.choice(sample_table, size=(len(positive_pairs), neg_count)).tolist()
    return neg_v

In [56]:
neg_v = get_negative_sampling(result_pairs,3)

In [57]:
neg_v

[[60360, 87336, 8658],
 [45008, 4792, 19654],
 [51493, 76622, 68416],
 [64869, 60789, 25258],
 [36741, 21429, 38853],
 [52013, 28271, 96883],
 [85242, 73450, 88603],
 [8019, 11007, 55982],
 [566, 34793, 62701],
 [9530, 77373, 43402],
 [70379, 64592, 7312],
 [91961, 86361, 91742],
 [95919, 5394, 25887],
 [84888, 56227, 28354],
 [71227, 40136, 34004],
 [7375, 96511, 5335],
 [63117, 52056, 93130],
 [30310, 75075, 29260],
 [10356, 8793, 22742],
 [61670, 85568, 4443],
 [39683, 59619, 31380],
 [46645, 64016, 50633],
 [90470, 28271, 94784],
 [54576, 97571, 58259],
 [80686, 82799, 4232],
 [70575, 42092, 88325],
 [89653, 93213, 28271],
 [85264, 58509, 37681],
 [75440, 76374, 14359],
 [32646, 86349, 80937],
 [86653, 56435, 94461],
 [56350, 32771, 68293]]